 ## Create the data

In [3]:

## Create the data

# import required modules
from mimesis.locales import Locale
from mimesis.keys import maybe
from mimesis.schema import Field, Schema
from mimesis import Datetime

from random import Random
from uuid import UUID, uuid4

from bench_utils import generate_uuid4, get_gen_uuid4_unique_list, table_definition

# create data

## person

person_id = generate_uuid4(table_definition['person_amount'], seed=10)

f = Field(locale=Locale.EN_GB, seed=10)

def person_generator() -> dict:
    first_name = f('first_name')
    last_name = f('last_name')
    return {
        "_id":next(person_id),
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.9)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }

person_schema = Schema(
    schema=person_generator,
    iterations=table_definition['person_amount']
)
person_data = person_schema.create()

person_id_count = range(table_definition['person_amount']-1)

print("person data created")

## artist

artist_id = generate_uuid4(table_definition['artist_amount'], seed=20)

f = Field(locale=Locale.EN_GB, seed=20)

def artist_generator() -> dict:
    first_name = f("first_name")
    last_name = f("last_name")
    return {
        "_id":next(artist_id), 
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.5)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }

artist_schema = Schema(
    schema=artist_generator,
    iterations=table_definition['artist_amount']
)

artist_data = artist_schema.create()

artist_id_count = range(table_definition['artist_amount']-1)

print("artist data created")

## product

product_id = generate_uuid4(table_definition['product_amount'], seed=30)

f = Field(locale=Locale.EN_GB, seed=30)
dt = Datetime(seed=30)

rnd = Random()
rnd.seed(30)

def product_generator() -> dict:
    created_at = dt.datetime(start=2023, end=2023)
    quantity = rnd.randint(1, 20)
    return {
        "_id":next(product_id),
        "name":' '.join(f('words', quantity=2)),
        "description":' '.join(f('words', quantity=rnd.randint(8, 25))),
        "category":f('choice', items=["oil paint", "watercolor", "acrylic paint", "charcoal", "pencil", "ink", "pastel", "collage", "digital art", "mixed media"]),
        "price":f('price', minimum=500, maximum=25000),
        "currency":f('currency_symbol'),
        "discount":f('float_number', start=0.2, end=0.8, precision=1, key=maybe(None, probability=0.8)),
        "quantity":quantity, 
        "image_url":f('stock_image_url'),
        "artist":artist_data[f('choice', items=artist_id_count)]['_id'],
        "creation_history": {
            "created_at":created_at,
            "quantity":quantity
        }
    }

product_schema = Schema(
    schema=product_generator,
    iterations=table_definition['product_amount']
)

product_data = product_schema.create()

product_id_count = range(table_definition['product_amount']-1)

print("product data created")

## order

order_id = generate_uuid4(table_definition['order_amount'], seed=40)

f = Field(locale=Locale.EN_GB, seed=40)
dt = Datetime(seed=40)

rnd = Random()
rnd.seed(40)

def order_generator() -> dict:
    person_number = f('choice', items=person_id_count)
    product_number = f('choice', items=product_id_count)
    shipping_address = person_data[person_number]['address']
    order_date = dt.datetime(start=2023, end=2023)
    return {
        "_id":next(order_id),
        "person":person_data[person_number]['_id'],
        "product":product_data[product_number]['_id'],
        "product_name":product_data[product_number]['name'],
        "currency":product_data[product_number]['currency'],
        "discount":product_data[product_number]['discount'],
        "price":product_data[product_number]['price'],
        "quantity": rnd.randint(1, 3),
        "order_date":order_date,
        "shipping_address":shipping_address,
        "payment_method":f('choice', items=['credit card','debit card', 'PayPal']),
        "order_status":f('choice', items=['pending','processing', 'shipped', 'delivered'], key=maybe(None, probability=0.1))
    }

order_schema = Schema(
    schema=order_generator,
    iterations=table_definition['order_amount']
)

order_data = order_schema.create()

print("order data created")

## review

review_id = generate_uuid4(table_definition['review_amount'], seed=50)

f = Field(locale=Locale.EN_GB, seed=50)
dt = Datetime(seed=50)

rnd = Random()
rnd.seed(50)

def review_generator() -> dict:
    review_date = dt.datetime(start=2023, end=2023)
    return {
        "_id":next(review_id),
        "person":person_data[f('choice', items=person_id_count)]['_id'],
        "product":product_data[f('choice', items=product_id_count)]['_id'],
        "artist":artist_data[f('choice', items=artist_id_count)]['_id'],
        "rating":f('choice', items=[1,2,3,4,5]),
        "review_text":' '.join(f('words', quantity=rnd.randint(8, 50))),
        "review_date": review_date
    }

review_schema = Schema(
    schema=review_generator,
    iterations=table_definition['review_amount']
)


review_data = review_schema.create()

print("review data created")


person data created
artist data created
product data created
order data created
review data created


In [ ]:
delete_10_uuids = get_gen_uuid4_unique_list(total_num=table_definition['review_amount'], list_num=10, seed=50)
delete_10_uuids

In [ ]:
delete_10_uuids.pop()

In [ ]:
[product_data[i]['artist'] for i in range(5)]
# ['artist:⟨db9fb768-148c-4388-9039-d1216cb6623f⟩',
#  'artist:⟨37cc1e86-2114-495c-9f58-c240c8f2b42f⟩',
#  'artist:⟨e857c13e-2f8d-41b0-86c9-423b775b26f3⟩',
#  'artist:⟨184719af-792c-4505-9a96-814f6f8b1d3a⟩',
#  'artist:⟨fa2f247f-e3ca-487b-a493-a3c2d13e66bf⟩']

In [ ]:
test_review_id_list = [review_data[i]["_id"] for i in range(10)]
test_review_id_list

In [ ]:
test_review = get_gen_uuid4_unique_list(10, 5, 50)

In [ ]:
test_review

In [ ]:
for i in test_review:
    if i in test_review_id_list:
        print("yes: ", i)
    else:
        print("no: ", i)

 ## Load the data

In [2]:
from pymongo import MongoClient
from bson.binary import UuidRepresentation
from datetime import datetime,timezone

MONGODB_URI = "mongodb://localhost:27017/"

client = MongoClient(MONGODB_URI, uuidRepresentation='standard')
db = client["surreal_deal"]

# Create collections
person = db["person"]
product = db["product"]
order = db["order"]
artist = db["artist"]
review = db["review"]


In [ ]:
# in case it exists drop the database
client.drop_database('surreal_deal')

In [4]:
# %%time
person.insert_many(person_data)


InsertManyResult([UUID('7b89296c-6dcb-4c50-8857-7eb1924770d3'), UUID('766bad07-34c2-4a80-83cc-0f2793fdcab8'), UUID('470b9805-d2d6-4877-bdc5-9a3ad035d259'), UUID('08ceac39-2904-4def-8f84-b683a749f9c5'), UUID('137a9777-53e8-4b43-bd76-3fb9854a9657'), UUID('bedc25e6-f3eb-4f12-b3d0-6f863fffc830'), UUID('6b9f15c4-0b68-4c1c-9c74-e45eff1e5bef'), UUID('5ae6a228-9a6a-4329-a381-23e5dc338383'), UUID('d3ac535f-489b-440f-abd7-f50361b0ee09'), UUID('2cb7362c-74f2-42ed-8327-79eeacca7f0d'), UUID('5cd2875e-a96e-42b3-8d98-4bffaf949e5e'), UUID('dc2c2e2c-c491-44d0-b4f9-42cb220adb0a'), UUID('708cc1b6-f829-429f-bd48-06c2fb7f6f5d'), UUID('953b00b0-0b54-4a22-a00f-ecc19d02fc90'), UUID('31e875ba-224c-4601-bc53-d0e30109c207'), UUID('894deab4-4d88-450f-a8da-c663f0e58650'), UUID('3d7c95f9-e5f0-407e-85a5-6d7e5dbbb7ce'), UUID('735c076b-8c8a-48b2-aaac-3142507a2560'), UUID('a7298c66-10a3-4558-b850-36de6f9fb997'), UUID('807da245-d814-4575-931e-c56c95a4d257'), UUID('f843bdb8-396b-483a-9798-c9cf280b11fd'), UUID('08208d09-0

In [ ]:
%%time
product.insert_many(product_data)


In [ ]:
%%time
order.insert_many(order_data)


In [ ]:
%%time
artist.insert_many(artist_data)


In [ ]:

review.insert_many(review_data)


 ## Run the queries

In [ ]:
from random import randint
# getting just an array of ids to use for loops in some queries
person_ids = person.distinct("_id")
product_ids = product.distinct("_id")
order_ids = order.distinct("_id")
artist_ids = artist.distinct("_id")
review_ids = review.distinct("_id")

# data too big error: Executor error during distinct command :: caused by :: distinct too big, 16mb cap, full error: {'ok': 0.0, 'errmsg': 'Executor error during distinct command :: caused by :: distinct too big, 16mb cap', 'code': 17217, 'codeName': 'Location17217'}


 ### Q1: lookup vs record links

In [ ]:
%%timeit
list(review.aggregate([
	{
		"$lookup": {
			"from": "artist",
			"localField": "artist",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
			],
			"as": "artist",
		}

	},
	{
		"$lookup": {
			"from": "person",
			"localField": "person",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
			],
			"as": "person",
		}

	},
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "category": 1, "price": 1 } }
			],
			"as": "product",
		}
	}
]))


 ### Q2: lookup vs graph

In [ ]:
# %%timeit
list(order.aggregate([
	{
		"$lookup": {
			"from": "person",
			"localField": "person",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
			],
			"as": "person",
		}
	},
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{
					"$lookup": {
						"from": "artist",
						"localField": "artist",
						"foreignField": "_id",
						"pipeline": [
							{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
						],
						"as": "artist",
					}
				},
				{ "$project": { "_id": 0, "category": 1, "description": 1, "image_url": 1, "artist": 1 } }
			],
			"as": "product",
		}
	},
	{ "$project": { "_id": 0, "price": 1, "order_date": 1, "product_name": 1, "artist": 1, "person": 1, "product": 1 } }
]))


In [ ]:
%%timeit
list(order.aggregate([
	{
		"$lookup": {
			"from": "person",
			"localField": "person",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
			],
			"as": "person",
		}
	},
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "category": 1, "image_url": 1, "description": 1,} }
			],
			"as": "product",
		}
	},
	{ "$project": { "_id": 0, "price": 1, "order_date": 1, "product_name": 1, "person": 1, "product": 1 } }
]))

In [ ]:
%%timeit
list(order.aggregate([
	{
		"$lookup": {
			"from": "person",
			"localField": "person",
			"foreignField": "_id",
			"as": "person",
		}
	},
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"as": "product",
		}
	},
	{ "$project": { "_id": 0, "price": 1, "order_date": 1, "product_name": 1, "person.name": 1, "person.email": 1, "person.phone": 1, "product.category": 1, "product.description": 1, "product.image_url": 1 } }
]))

In [ ]:
%%timeit
list(order.aggregate([
    {
        '$graphLookup': {
            'from': 'person', 
            'startWith': '$person', 
            'connectFromField': 'person', 
            'connectToField': '_id', 
            'maxDepth': 0, 
            'as': 'person'
        }
    }, 
    {
        '$graphLookup': {
            'from': 'product', 
            'startWith': '$product', 
            'connectFromField': 'product', 
            'connectToField': '_id', 
            'maxDepth': 0, 
            'as': 'product'
        }
    }, 
    { "$project": { "_id": 0, "price": 1, "order_date": 1, "product_name": 1, "person.name": 1, "person.email": 1, "person.phone": 1, "product.category": 1, "product.description": 1, "product.image_url": 1 } }
]))

 ### Q3: Name and email for all customers in England

In [ ]:
%%time
person.create_index({"address.country": 1})

In [ ]:
%%timeit
list(person.find(
	{ "address.country": "England" },
	{ "_id": 0, "name": 1, "email": 1 }
))


 ### Q4: Count the number of confirmed orders in Q1 by artists in England

In [ ]:
list(order.aggregate([
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{
					"$lookup": {
						"from": "artist",
						"localField": "artist",
						"foreignField": "_id",
						"pipeline": [
							{ "$project": { "_id": 0, "address.country": 1 } }
						],
						"as": "artist",
					}
				},
				{ "$project": { "_id": 0, "artist": 1 } }
			],
			"as": "product",
		}
	},
	{
		"$match": {
            "order_status": { "$in": ['delivered', 'processing', 'shipped'] },
            "$expr": { "$lt": ["$order_date", datetime(2023, 4, 1, 0, 0, 0, 0)]},
			"product.artist.address.country": "England",
		}
	},
	{ "$count": "count" }
]))


In [ ]:
list(order.aggregate([
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{
					"$lookup": {
						"from": "artist",
						"localField": "artist",
						"foreignField": "_id",
						"pipeline": [
							{ "$project": { "_id": 0, "address.country": 1 } }
						],
						"as": "artist",
					}
				},
				{ "$project": { "_id": 0, "artist": 1 } }
			],
			"as": "product",
		}
	},
	{
		"$match": {
			"product.artist.address.country": "England"
		}
	},
	{ "$count": "count" }
]))

In [ ]:
list(order.aggregate([
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{
					"$lookup": {
						"from": "artist",
						"localField": "artist",
						"foreignField": "_id",
						"pipeline": [
							{ "$project": { "_id": 1, "address.country": 1 } }
						],
						"as": "artist",
					}
				},
				{ "$project": { "_id": 0, "artist": 1 } }
			],
			"as": "product",
		}
	},
	{
		"$match": {
			"product.artist.address.country": "England"
		}
	},
	{ "$project": { "_id": 0, "product.artist._id": 1} }
]))

In [ ]:
list(product.aggregate([
        {
			"$lookup": {
				"from": "artist",
				"localField": "artist",
				"foreignField": "_id",
				"pipeline": [
					{ "$project": { "_id": 0, "address.country": 1 } }
				],
				"as": "artist",
			}
		},
	{
		"$match": {
			"artist.address.country": "England"
		}
	},
	{ "$count": "count" }
]))

In [ ]:
product.distinct("artist")

In [ ]:
list(product.aggregate([
        {
			"$lookup": {
				"from": "artist",
				"localField": "artist",
				"foreignField": "_id",
				"pipeline": [
					{ "$project": { "_id": 1, "address.country": 1 } }
				],
				"as": "artist",
			}
		},
	{
		"$match": {
			"artist.address.country": "England"
		}
	},
	{ "$project": { "_id": 0, "artist._id": 1} }
]))

In [ ]:
list(product.aggregate([
        {
			"$lookup": {
				"from": "artist",
				"localField": "artist",
				"foreignField": "_id",
				"pipeline": [
					{ "$project": { "_id": 1, "address.country": 1 } }
				],
				"as": "artist",
			}
		},
	{ "$project": { "_id": 0, "artist._id": 1, "artist.address.country":1} }
]))

In [ ]:
sourceFile = open('demo.txt', 'w')
print(
list(product.aggregate([
        {
			"$lookup": {
				"from": "artist",
				"localField": "artist",
				"foreignField": "_id",
				"pipeline": [
					{ "$project": { "_id": 1, "address.country": 1 } }
				],
				"as": "artist",
			}
		},
	{
		"$match": {
			"artist.address.country": "England"
		}
	},
	{ "$project": { "_id": 0, "artist._id": 1} }
]))
, file = sourceFile)
sourceFile.close()

In [ ]:
sourceFile = open('demo.txt', 'w')
print('Hello, Python!', file = sourceFile)
sourceFile.close()

In [ ]:
len(list(product.aggregate([
        {
			"$lookup": {
				"from": "artist",
				"localField": "artist",
				"foreignField": "_id",
				"pipeline": [
					{ "$project": { "_id": 1, "name": 1, "address.country": 1 } }
				],
				"as": "artist",
			}
		},
	{
		"$match": {
            "artist._id": { "$nin": SDB_artists }
		}
	},
	{ "$project": { "_id": 1, "artist": 1} }
])))

In [ ]:
product.count_documents({
})

In [ ]:
artist.count_documents({
	"address.country": "England"
})

In [ ]:
order.distinct("order_status", {"$and": [{ "order_status": { "$ne": None } }, { "order_status": { "$ne": "pending" } }]})

In [ ]:
order.distinct("order_status", {"order_status": { "$nin": ["pending", None] }})

In [ ]:
%%timeit
order.count_documents({
    "$expr": { "$lte": [{ "$month": "$order_date" }, 3] },
	"order_status": { "$nin": ["pending", None] }
})

In [ ]:
order.create_index({"order_date": 1, "order_status": 1})

In [ ]:
order.create_index({"order_status": 1, "order_date": 1})

In [ ]:
%%timeit
order.count_documents({
	"order_status": { "$in": ['delivered', 'processing', 'shipped'] },
	"$expr": { "$lte": [{ "$month": "$order_date" }, 3] }
})

In [ ]:
%%timeit
order.count_documents({
	"order_status": { "$in": ['delivered', 'processing', 'shipped'] },
	"$expr": { "$lt": ["$order_date", datetime(2023, 4, 1, 0, 0, 0, 0)]}
})

In [ ]:
%%timeit
order.estimated_document_count({
	"order_status": { "$in": ['delivered', 'processing', 'shipped'] },
	"$expr": { "$lt": ["$order_date", datetime(2023, 4, 1, 0, 0, 0, 0)]}
})

In [ ]:
order.count_documents({
	"$and": [{ "order_status": { "$ne": None } }, { "order_status": { "$ne": "pending" } }]
})

In [ ]:
order.count_documents({
	"$expr": { "$lte": [{ "$month": "$order_date" }, 3] }
})

In [ ]:
list(order.find(
	{"$expr": { "$lte": [{ "$month": "$order_date" }, 3] }},
	{ "_id":0, "order_date": { "$month": "$order_date" } }
))

 ### Q5: Delete a specific review

In [ ]:
%%time
review.delete_one({ "_id": UUID('3e519815-ee9f-46d1-b134-7478ca4a761e') })


In [ ]:
delete_uuid = [
    UUID('3e4b5caf-c52a-4010-b664-bddcc7767168'),
    UUID('3e519815-ee9f-46d1-b134-7478ca4a761e'),
    UUID('3e5e8004-32da-499c-b7b2-5dc9caa0ae8f'),
    UUID('3e748d02-c70d-4955-9fdd-c43553e30b35'),
    UUID('3e886e88-abd9-4923-8a57-8389799c1840'),
    UUID('3ea31852-8979-4cb3-bfa1-035aa99de7a2'),
    UUID('3ebde78a-7b41-42ac-b7e8-7f9ec7ba6ceb'),
    UUID('3ec29563-1990-4f51-be0c-613aeb1ecab6'),
    UUID('3f2b0773-6df7-4331-b848-1c3a6e3149d6'),
    UUID('3f4c2391-b00c-4246-b277-00d4d3770a56')
]

In [ ]:
review.distinct("_id")[495:505]

In [ ]:
product.count_documents({})

In [ ]:
[rnd.randint(0, max(review_id_count)) for _ in range(10) ]

In [ ]:
rnd.randint(1, max(review_id_count))

In [ ]:
review_id

In [ ]:
review.distinct("_id", {"_id": UUID('00482e41-4c1c-4fca-bb74-2433fab5ec20')})

In [ ]:
db.product.distinct("discount", { "_id" : random_product_id })

In [ ]:
review.find_one({},{"_id": rnd.randint(0, max(review_id_count))})

In [ ]:
%%time
review.delete_one({ "_id": delete_uuid.pop() })

In [ ]:
review.delete_one({ "_id": UUID('7eb18c1f-1bdf-42fe-b033-1c4c94485481') })

 ### Q6: Delete reviews from a particular category

In [ ]:
%%timeit
product.distinct("_id", { "category": "charcoal" })

In [ ]:
%%time
review.delete_many({ "product": { "$in": product.distinct("_id", { "category": "charcoal" }) } })


 ### Q7: Update a customer address

In [ ]:
%%timeit
person.update_one(
	{ "_id": person_ids[randint(0, person_id_count)] },
	{
		"$set": {
			"address": {
				'address_line_1': '497 Ballycander',
				'address_line_2': "null",
				'city': 'Bromyard',
				'country': 'Wales',
				'post_code': 'ZX8N 4VJ',
				'coordinates': [68.772592, -35.491877]
			}
		}
	}
)


 ### Q8: Update discounts for products

In [ ]:
product.create_index({"price": 1})

In [ ]:
%%time
product.update_many(
	{ "price": { "$lt": 1000 } },
	{ "$set": { "discount": 0.2 } }
)

 ### Q9: "Transaction"* order from a new customer

In [ ]:
%%timeit
# Transaction - order from a new customer

new_person_id = uuid4()
random_product_id = product_ids[randint(0, product_id_count)]

# insert into the person table
person.insert_one({
		'_id': new_person_id,
		'first_name': 'Karyl',
		'last_name': 'Langley',
		'name': 'Karyl Langley',
		'company_name': "null",
		'email': 'dee1961@gmail.com',
		'phone': '+44 47 3516 5895',
		'address': {
			'address_line_1': '510 Henalta',
			'address_line_2': "null",
			'city': 'Lyme Regis',
			'country': 'Northern Ireland',
			'post_code': 'TO6Q 8CM',
			'coordinates': [-34.345071, 118.564172]
		}
	})

# relate into the order table
order.insert_one({
	"_id": uuid4(),
	"person": new_person_id,
	"product": random_product_id,
	'currency': '£',
	'discount': db.product.distinct("discount", { "_id" : random_product_id }), 
	"order_date": datetime.now(tz=timezone.utc),
	"order_status": "pending",
	"payment_method": "PayPal",
	"price": db.product.distinct("price", { "_id" : random_product_id }),
	"product_name": db.product.distinct("name", { "_id" : random_product_id }),
	"quantity": 1,
	"shipping_address": db.person.distinct("address", { "_id" : new_person_id })
})

# update the product table to reduce the quantity
product.update_one(
	{ "_id": random_product_id },
	{ "$inc": { "quantity": -1 } }
)


 ### Q10: "Transaction"* - New Artist creates their first product

In [ ]:
%%timeit
# Transaction - New Artist creates their first product

new_artist_id = uuid4()

# insert into the artist table
artist.insert_one({
    '_id': new_artist_id,
    'first_name': 'Anderson',
    'last_name': 'West',
    'name': 'Anderson West',
    'company_name': 'Atkins(ws) (ATK)',
    'email': 'six1933@gmail.com',
    'phone': '056 5881 1126',
    'address': {
        'address_line_1': '639 Connaugh',
        'address_line_2': "null",
        'city': 'Ripon',
        'country': 'Scotland',
        'post_code': 'CG3U 4TH',
        'coordinates': [4.273648, -112.907273]
	}
})

# insert into the product table
product.insert_one({
    '_id': uuid4(),
    'name': 'managed edt allocated pda',
    'description': 'counseling dildo greek pan works interest xhtml wrong dennis available cl specific next tower webcam peace magic',
    'category': 'watercolor',
    'price': 15735.96,
    'currency': '£',
    'discount': "null",
    'quantity': 1,
    'image_url': 'https://source.unsplash.com/1920x1080?',
    "artist": new_artist_id,
    "creation_history": {
        "quantity": 1,
        "created_at": datetime.now(tz=timezone.utc)
	}
})

In [ ]:
client.close()